In [24]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [25]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [26]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [27]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [28]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [29]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [30]:
## 
n='Input_A3_017' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A3_017
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.009448426063746636
test_rmse: 0.009560542910671769
test_r2: 0.4792472812657049
feature number: 3
train_rmse: 0.009205584329713887
test_rmse: 0.009355696027864895
test_r2: 0.5006511239690108
feature number: 4
train_rmse: 0.009110841653740622
test_rmse: 0.00927883664383767
test_r2: 0.5094779890548219
feature number: 5
train_rmse: 0.009021544601542492
test_rmse: 0.009185245312196513
test_r2: 0.5194517040383431
feature number: 6
train_rmse: 0.00900483950955693
test_rmse: 0.009203696984551929
test_r2: 0.5174761800487577
feature number: 7
train_rmse: 0.008961102956302812
test_rmse: 0.009208633165954964
test_r2: 0.5168433754335673
feature number: 8
train_rmse: 0.008959389068964114
test_rmse: 0.009223382322211112
test_r2: 0.5153642238161051
feature number: 9
train_rmse: 0.008941102618453324
test_rmse: 0.009257937978939553
test_r2: 0.5117355726906383
feature number: 10
train_rmse: 0.008923969577419885
test_

train_rmse: 0.008883757350174463
test_rmse: 0.009400884542760463
test_r2: 0.49691905839661565
feature number: 15
train_rmse: 0.00887204281163348
test_rmse: 0.009416257836575095
test_r2: 0.49528157674258555
feature number: 16
train_rmse: 0.008851141584110364
test_rmse: 0.009447359932941048
test_r2: 0.4920349760493302
feature number: 17
train_rmse: 0.008841286519656634
test_rmse: 0.009488464187343514
test_r2: 0.4875367270661486
feature number: 18
train_rmse: 0.00882342932616345
test_rmse: 0.009493260208251253
test_r2: 0.48692320190304594
feature number: 19
train_rmse: 0.00881759228988293
test_rmse: 0.009526726174271768
test_r2: 0.48327018611120487
feature number: 20
train_rmse: 0.00874922595834042
test_rmse: 0.009556799163815344
test_r2: 0.4800774736920171
Test_rmse_min = 1.fea_num: 5  2.rmse: 0.009185245312196513
Test_r2_max = 1. fea_num: 5  2.r2: 0.5194517040383431
./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.009448426063746636
test_rmse: 0.0095605

train_rmse: 0.00900483950955693
test_rmse: 0.009203696984551929
test_r2: 0.5174761800487577
feature number: 7
train_rmse: 0.008961102956302812
test_rmse: 0.009208633165954964
test_r2: 0.5168433754335673
feature number: 8
train_rmse: 0.008959389068964114
test_rmse: 0.009223382322211112
test_r2: 0.5153642238161051
feature number: 9
train_rmse: 0.008941102618453324
test_rmse: 0.009257937978939553
test_r2: 0.5117355726906383
feature number: 10
train_rmse: 0.008923969577419885
test_rmse: 0.009296639729868657
test_r2: 0.5078325723394386
feature number: 11
train_rmse: 0.008897007319285406
test_rmse: 0.009309382589808332
test_r2: 0.506550658700678
feature number: 12
train_rmse: 0.00889371498550903
test_rmse: 0.00932915548214786
test_r2: 0.5045055141945846
feature number: 13
train_rmse: 0.0088891126714659
test_rmse: 0.009361623055272161
test_r2: 0.5010848568409433
feature number: 14
train_rmse: 0.008883757350174463
test_rmse: 0.009400884542760463
test_r2: 0.49691905839661565
feature number: 15


# 可以修改第一層的神經元數量

In [31]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(7, activation='sigmoid',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [32]:
fea_num=[4,5,6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.009716367140104097
[0.012033356030484055, 0.00841586143803087, 0.010530545552798377, 0.008866226999844436, 0.010714137414404289, 0.009687362847323775, 0.009975875159920374, 0.009834669478131131, 0.00934362769051216, 0.007762008789591508]


test_r2_ave: 0.44656350447773085
[0.26077558366528153, 0.5963538044699224, 0.45850393120620214, 0.5656196372400184, 0.3932857041675324, 0.29482948242911866, 0.4053644532288252, 0.3641433778713281, 0.5153318742392613, 0.6114271962598188]


feature number: 5
test_rmse_ave: 0.009630495243165083
[0.010467441138507224, 0.009651634019819407, 0.00985628397353453, 0.009511525622377984, 0.011225125320301755, 0.008257762380234712, 0.01036299259036488, 0.008994981911691428, 0.008748215494286433, 0.00922898998053247]


test_r2_ave: 0.46093209150515413
[0.44064976462855665, 0.4691089527396207, 0.5256269794210604, 0.500088812992348, 0.3340337805380782, 0.4876015864980967, 0.35831892438189616, 0.46808743474748116, 0.57513363210537

In [ ]:
fea_num=[4,5,6,7,8,9]   # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.009545790864406403
[0.012635023844288197, 0.00882505537040899, 0.010325489111708589, 0.008775195677307281, 0.009838485753171058, 0.009033288587941, 0.009955187017459522, 0.00891971773868123, 0.009074532941148974, 0.008075932601949179]


test_r2_ave: 0.4677264478771268
[0.18500507940679112, 0.556147583784504, 0.47938721168162457, 0.5744935866616421, 0.4884049590153412, 0.3868386344433201, 0.4078282268337061, 0.47695159270339405, 0.5428465810353431, 0.5793610232056012]


feature number: 5
test_rmse_ave: 0.009804409409887008
[0.010506706061429635, 0.009514954950054005, 0.010644885095186934, 0.008909387286499017, 0.010148157911304748, 0.009754554730984324, 0.01138033175592269, 0.009219875321668113, 0.009605233930865036, 0.008360007054955574]


test_r2_ave: 0.4373615468914619
[0.43644548269787176, 0.484038634633572, 0.4466810788289043, 0.5613802661919995, 0.45569260222623, 0.28501338349605676, 0.22614658871081839, 0.4411570745523813, 0.48781211482733466, 0

In [ ]:
fea_num=[4,5,6,7,8,9]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7,8,9]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7,8,9]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[4,5,6,7,8,9] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')